#### This document used for downloading data from the New York Times webpages

This programme can download data(i.e. news title) for the whole specific year from the nytimes news website. <br>

Change the code where says **# CHANGE HERE**

In [1]:
import os
import pandas as pd
import requests
import json
import time
import dateutil
import datetime
import configparser
from dateutil.relativedelta import relativedelta
import urllib3
from nytimesarticle import articleAPI
from pprint import pprint
from bs4 import BeautifulSoup  
import re
from tqdm import tqdm

#### This is the API 
If the API is invalid, please apply a new one from the website via https://developer.nytimes.com/apis

In [2]:

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
YOUR_API_KEY= 'VCdDP0EMPdz3AUYHAkCQ12h6ELuTzJO8'


#### Change the date here to get the urls you want 

In [3]:
# CHANGE HERE
end = datetime.date(2013, 1, 1)
start = datetime.date(2012, 1, 1)
print('Start date: ' + str(start))
print('End date: ' + str(end))
def date_format(start, end):
    dates = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime('%Y %m').tolist()]
#     print(dates)
    months_in_range = []
    for date in dates:
        #print(date[1][0])
        if date[1][0]=='0':
            months_in_range.append([date[0],date[1].replace('0','')])
        else:
            months_in_range.append([date[0],date[1]])
    return months_in_range
months_in_range=date_format(start,end)
months_in_range

Start date: 2012-01-01
End date: 2013-01-01


[['2012', '1'],
 ['2012', '2'],
 ['2012', '3'],
 ['2012', '4'],
 ['2012', '5'],
 ['2012', '6'],
 ['2012', '7'],
 ['2012', '8'],
 ['2012', '9'],
 ['2012', '10'],
 ['2012', '11'],
 ['2012', '12'],
 ['2013', '1']]

In [4]:
def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + YOUR_API_KEY
    try:
        response = requests.get(url, verify=False).json()
    except Exception:
        return None
    time.sleep(6)
    return response


def is_valid(article, date):
    '''An article is only worth checking if it is in range, and has a headline.'''
    is_in_range = date > start and date < end
    has_headline = type(article['headline']) == dict and 'main' in article['headline'].keys()
    return is_in_range and has_headline


def parse_response(response):
    '''Parses and returns response as pandas data frame.'''
    data = {'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'section': [],
        'keywords': []}
    
    articles = response['response']['docs'] 
    urls = []
    dates_url = []
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        if is_valid(article, date):
            time_stamp = pd.Timestamp(article['pub_date'])
            dates_url.append(time_stamp.strftime("%Y%m%d-%H%M%S"))
            urls.append(article["web_url"])
    print("Parse articles : " , len(urls))
    return urls,dates_url


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    if not os.path.exists('nytimes_data'):
        os.mkdir('nytimes_data')
    for date in dates:
        print('Working on ' + str(date) + '...')
#         csv_path = 'headlines/' + date[0] + '-' + date[1] + '.csv'
        txt_path = "nytimes_data/"+ date[0] + '_' + date[1]
#         if not os.path.exists(txt_path): # If we don't already have this month 
        response = send_request(date)
#         if response is not None:
        urls, date_all = parse_response(response)
#         print(date_all)
        total += len(urls)
        counter = 0
        directory = "nytimes_data/"+date[0]+"/"+date[1]
        if not os.path.exists(directory):
            os.makedirs(directory)
        for every_url in tqdm(urls):
            
            file = open(directory+"/"+ date_all[counter] + "_"+str(counter)+".txt","w",encoding='utf-8') 
            r = requests.get(every_url)
#             time.sleep(6)
            soup = BeautifulSoup(r.text, 'html.parser')  
            results = soup.find_all('p')
            for para in results:
                data = re.sub(r'[^\w\s]',' ',para.text) 
                file.write(data)
                file.write("\n")
            file.close()
            counter+=1
        print('Saving ' + txt_path + '...')
        pprint(urls)
                
#                 df.to_csv(csv_path, index=False)
        
    print('Number of articles collected: ' + str(total))

#### get requests and save data in a local folder
For each webpage, it will save the content into one 'txt' file, named by its published date and time

In [5]:
get_data(months_in_range)

Date range: ['2012', '1'] to ['2013', '1']
Working on ['2012', '1']...


  0%|                                                                                 | 2/9037 [00:00<08:52, 16.96it/s]

Parse articles :  9037


  0%|▎                                                                             | 36/9037 [00:15<1:03:58,  2.34it/s]


KeyboardInterrupt: 